In [1]:
import numpy as np

def estimateCovariance(df):
  #compute mean of feature
  mean = df.select(df["features"]).map(lambda x: x[0]).mean()
  dfZeromean = df.select(df["features"]).map(lambda x:x[0]).map(lambda x:x-mean)
  return dfZeromean.map(lambda x:np.outer(x,x)).sum()/df.count()

In [2]:
#pca function writting
from numpy.linalg import eigh
from pyspark.mllib.linalg import *

def pca(df, k=2):
  cov = estimateCovariance(df)
  col = cov.shape[1]
  eigVals, eigVecs = eigh(cov)
  inds = np.argsort(eigVals)
  eigVecs = eigVecs.T[inds[-1:-(col+1):-1]]
  components = eigVecs[0:k]
  eigVals = eigVals[inds[-1:-(col+1):-1]]
  score = df.select(df['features']).map(lambda x: x[0]).map(lambda x: np.dot(x, components.T) )
  scoreDF = sqlContext.createDataFrame(score.map(lambda x: (DenseVector(x),)), ['pca_features'])
  # Return the `k` principal components, `k` scores, and all eigenvalues
     
  return components.T, scoreDF, eigVals

In [3]:
from pyspark.ml.feature import PCA
from pyspark.mllib.linalg import *
data = [(Vectors.dense([0.0, 1.0, 0.0, 7.0, 0.0]),),
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = sqlContext.createDataFrame(data,["features"])
pca_extracted = PCA(k=2, inputCol="features", outputCol="pca_features")
model = pca_extracted.fit(df)
df_pca = model.transform(df)
df_pca.collect()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o874.fit.
: java.lang.IllegalArgumentException: requirement failed: Column features must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually struct<type:tinyint,size:int,indices:array<int>,values:array<double>>.
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.util.SchemaUtils$.checkColumnType(SchemaUtils.scala:43)
	at org.apache.spark.ml.feature.PCAParams$class.validateAndTransformSchema(PCA.scala:57)
	at org.apache.spark.ml.feature.PCA.validateAndTransformSchema(PCA.scala:71)
	at org.apache.spark.ml.feature.PCA.transformSchema(PCA.scala:106)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:95)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-2285154199981497> in <module> 
 6 df = sqlContext . createDataFrame ( data , [ "features" ] ) 
 7 pca_extracted = PCA ( k = 2 , inputCol = "features" , outputCol = "pca_features" ) 
 ----> 8 model = pca_extracted . fit ( df ) 
 9 df_pca = model . transform ( df ) 
 10 df_pca . collect ( ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 77 raise QueryExecutionException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 78 if s . startswith ( 'java.lang.IllegalArgumentException: ' ) : 
 ---> 79 raise IllegalArgumentException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 80 raise 
 81 return deco

 IllegalArgumentException : 'requirement failed: Column features must be of type struct<type:tinyint,size:int,indices:

In [4]:
comp, score, eigVals = pca(df)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2285154199981498> in <module> 
 ----> 1 comp , score , eigVals = pca ( df ) 

 <command-2285154199981496> in pca (df, k) 
 4 
 5 def pca ( df , k = 2 ) : 
 ----> 6 cov = estimateCovariance ( df ) 
 7 col = cov . shape [ 1 ] 
 8 eigVals , eigVecs = eigh ( cov ) 

 <command-2285154199981495> in estimateCovariance (df) 
 3 def estimateCovariance ( df ) : 
 4 #compute mean of feature 
 ----> 5 mean = df . select ( df [ "features" ] ) . map ( lambda x : x [ 0 ] ) . mean ( ) 
 6 dfZeromean = df . select ( df [ "features" ] ) . map ( lambda x : x [ 0 ] ) . map ( lambda x : x - mean ) 
 7 return dfZeromean . map ( lambda x : np . outer ( x , x ) ) . sum ( ) / df . count ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1332 if name not in self . columns : 
 1333 raise AttributeError(
 -> 1334 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1335 jc = self . _jdf . apply ( name ) 
 1336 return Column ( jc ) 

 AttributeError : 'DataFrame' object has no attribute 'map'